In [ ]:
import streamlit as st
import pandas as pd 
import folium 
from visualization_route import plot_route
from streamlit_folium import st_folium

import importlib
import simulator
importlib.reload(simulator)
from simulator import Simulation



sim = Simulation()
total_route = sim.run()

df = pd.read_excel("정류장_좌표 (1).xlsx")

In [153]:
plot_df.head()

,Unnamed: 0,정류장_ID,정류장,x,y
4,4,04_오이도종합어시장,오이도종합어시장,126.689667,37.345000
5,5,05_함상전망대,함상전망대,126.688517,37.341817
17,17,17_홈플러스,홈플러스,126.725683,37.344417
21,21,21_이마트,이마트,126.737283,37.346050
30,30,30_이마트,이마트,126.737283,37.346050


In [ ]:
total_x = []
total_y = []
plot_df = pd.DataFrame()
# 여기서 total_route에 시간을 넣었음 
# 스트림릿에서 시간을 골라서 바로 넣게끔 하고픔 
for station in total_route['14']: 
  plot_df = pd.concat([plot_df, df[df['정류장_ID'] == station.split('(')[0]]],axis= 0)

In [152]:
import osmnx as ox
import folium
import pandas as pd

# 정류장 DataFrame 예시 (이미 plot_df라고 가정)
# plot_df에는 '정류장_ID', 'x'(경도), 'y'(위도) 컬럼이 있음

# 1. 지도 중심 계산
center_lat = plot_df["y"].mean()
center_lon = plot_df["x"].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# 2. osmnx로 도로망 그래프 불러오기 (반경 3km 내 도로망)
G = ox.graph_from_point((center_lat, center_lon), dist=3000, network_type="drive")

# 3. 정류장 경로 순서대로 실제 도로 기반 연결
for i in range(len(plot_df) - 1):
    origin = (plot_df.iloc[i]["y"], plot_df.iloc[i]["x"])
    destination = (plot_df.iloc[i + 1]["y"], plot_df.iloc[i + 1]["x"])
    
    try:
        node_origin = ox.nearest_nodes(G, origin[1], origin[0])
        node_dest = ox.nearest_nodes(G, destination[1], destination[0])
        shortest_path = ox.shortest_path(G, node_origin, node_dest, weight="length")
        path_coords = [(G.nodes[n]["y"], G.nodes[n]["x"]) for n in shortest_path]

        folium.PolyLine(path_coords, color="blue", weight=5, opacity=0.8).add_to(m)
    except Exception as e:
        print(f"🚫 경로 계산 실패 ({plot_df.iloc[i]['정류장_ID']} → {plot_df.iloc[i+1]['정류장_ID']}): {e}")

# 4. 각 정류장에 마커 추가
for _, row in plot_df.iterrows():
    folium.Marker(
        location=[row["y"], row["x"]],
        popup=row["정류장_ID"],
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(m)

# 5. 지도 출력
m
